# Sufficient Statistic Bias Calculations Normal 1 parameter known variance ANY PRIOR

## Hypothesis

### Hypothesis Bias Against

In [1]:
import numpy as np
from scipy.stats import norm
from scipy.special import logsumexp

def ss_hypothesis_bias_against_normal_1p(
    n_sample,
    hypothesis,
    n_datasets=1000,
    n_marginal=1000,
    sigma=1,
    prior=None,
    cumulative=False
):
    """
    Calculate the bias against a hypothesis using the sufficient statistic approach with an arbitrary prior.

    Parameters:
    - n_sample (int): Sample size for each dataset.
    - hypothesis (float): The hypothesized value of theta.
    - n_datasets (int): Number of datasets to simulate.
    - n_marginal (int): Number of samples to estimate the marginal likelihood.
    - sigma (float): Known standard deviation of the data.
    - prior (scipy.stats distribution): Prior distribution object (frozen) from scipy.stats.
    - cumulative (bool): Whether to return cumulative bias against values.

    Returns:
    - float or np.ndarray: Final bias against estimate or cumulative bias against values.
    """
    # Set default prior if none is provided (standard normal)
    if prior is None:
        prior = norm(loc=0, scale=1)

    # Step 1: Generate sample means x̄ from the sampling distribution under the hypothesis
    se = sigma / np.sqrt(n_sample)  # Standard error of the mean
    x_bar = np.random.normal(hypothesis, se, size=n_datasets)

    # Step 2: Compute the log-likelihood of x̄ under the hypothesis
    log_likelihood = norm.logpdf(x_bar, loc=hypothesis, scale=se)

    # Step 3: Sample theta from the prior distribution
    theta_samples = prior.rvs(size=n_marginal)

    # Step 4: Compute the marginal likelihood p(x̄) via Monte Carlo integration
    # Compute log p(x̄ | theta) for each theta_sample and x̄
    # Shape of log_likelihoods: (n_datasets, n_marginal)
    log_likelihoods = norm.logpdf(
        x_bar[:, np.newaxis],
        loc=theta_samples[np.newaxis, :],
        scale=se
    )

    # Compute log marginal likelihood for each x̄ using log-sum-exp trick
    # log p(x̄) = log(1/n_marginal * sum_{j} exp(log p(x̄ | theta_j)))
    log_marginal_likelihood = logsumexp(log_likelihoods, axis=1) - np.log(n_marginal)

    # Step 5: Compute the bias against
    bias_against = (log_likelihood - log_marginal_likelihood) <= 0

    # Compute cumulative bias against if requested
    bias_against_cumsum = np.cumsum(bias_against) / np.arange(1, n_datasets + 1)

    if cumulative:
        return bias_against_cumsum
    else:
        return bias_against_cumsum[-1]


#### Example: Hypothesis Bias Against

In [45]:
from scipy.stats import uniform

# Parameters
n_sample = 50
hypothesis = 0
n_datasets = 1000
n_marginal = 1000
sigma = 1

# Define a uniform prior over the interval [-2, 2]
uniform_prior = uniform(loc=-2, scale=4)  # loc is the start, scale is the width

# Calculate bias against using the uniform prior
bias_against_uniform = ss_hypothesis_bias_against_normal_1p(
    n_sample=n_sample,
    hypothesis=hypothesis,
    n_datasets=n_datasets,
    n_marginal=n_marginal,
    sigma=sigma,
    prior=uniform_prior,
    cumulative=False
)

# Calculate bias against using standard normal prior
bias_against_uniform = ss_hypothesis_bias_against_normal_1p(
    n_sample=n_sample,
    hypothesis=hypothesis,
    n_datasets=n_datasets,
    n_marginal=n_marginal,
    sigma=sigma,
    prior=None,
    cumulative=False
)


print(f"Bias against the hypothesis with uniform prior: {bias_against_uniform}")
print(f"Bias against the hypothesis with standard normal prior: {bias_against_uniform}")


Bias against the hypothesis with uniform prior: 0.041
Bias against the hypothesis with standard normal prior: 0.041


### Hypothesis Bias In Favor

In [46]:
import numpy as np
from scipy.stats import norm
from scipy.special import logsumexp

def ss_hypothesis_bias_in_favor_normal_1p(
    n_sample,
    hypothesis,
    delta,
    n_datasets=1000,
    n_marginal=1000,
    sigma=1,
    prior=None,
    cumulative=False
):
    """
    Estimates the bias in favor of a normal hypothesis using the sufficient statistic approach with an arbitrary prior.
    
    Parameters:
    ----------
    n_sample : int
        Sample size for each dataset (number of observations).
    hypothesis : float
        The hypothesized value of theta (theta_0).
    delta : float
        The specified distance delta.
    n_datasets : int, optional
        Number of datasets (sample means) to simulate (default is 1000).
    n_marginal : int, optional
        Number of samples to estimate the marginal likelihood (default is 1000).
    sigma : float, optional
        Known standard deviation of the data (default is 1).
    prior : scipy.stats distribution, optional
        Prior distribution object (frozen) from scipy.stats. Default is standard normal.
    cumulative : bool, optional
        If True, returns the cumulative bias in favor (default is False).
    
    Returns:
    -------
    float or np.ndarray
        Final bias in favor estimate or cumulative bias in favor values.
    """
    # Set default prior if none is provided (standard normal)
    if prior is None:
        prior = norm(loc=0, scale=1)

    # Standard error of the mean
    se = sigma / np.sqrt(n_sample)
    
    # Initialize to store results for +delta and -delta cases
    bias_in_favor_cumsums = []
    
    # Loop over +delta and -delta shifts
    for shift in [+delta, -delta]:
        # Step 1: Set theta = hypothesis + delta or hypothesis - delta
        theta = hypothesis + shift

        # Step 2: Generate sample means x̄ from p(x̄ | theta)
        x_bar = np.random.normal(theta, se, size=n_datasets)
        
        # Step 3: Compute log-likelihoods of x̄ under the hypothesis
        log_likelihood = norm.logpdf(x_bar, loc=hypothesis, scale=se)
        
        # Step 4: Sample theta_j from the prior distribution
        theta_j_samples = prior.rvs(size=n_marginal)
        
        # Step 5: Compute the marginal likelihood p(x̄) via Monte Carlo integration
        # Compute log p(x̄ | theta_j) for each theta_j and x̄
        # Shape of log_likelihoods: (n_datasets, n_marginal)
        log_likelihoods = norm.logpdf(
            x_bar[:, np.newaxis],
            loc=theta_j_samples[np.newaxis, :],
            scale=se
        )
        
        # Compute log marginal likelihood for each x̄ using log-sum-exp trick
        # log p(x̄) = log(1/n_marginal * sum_{j} exp(log p(x̄ | theta_j)))
        log_marginal_likelihood = logsumexp(log_likelihoods, axis=1) - np.log(n_marginal)
        
        # Step 6: Compute the log relative belief ratio for each x̄
        log_rb = log_likelihood - log_marginal_likelihood
        
        # Step 7: Determine bias in favor: RB(theta_0 | x̄) > 1
        bias_in_favor = (log_rb > 0).astype(int)
        
        # Calculate cumulative bias in favor (running mean)
        bias_in_favor_cumsum = np.cumsum(bias_in_favor) / np.arange(1, len(bias_in_favor) + 1)
        bias_in_favor_cumsums.append(bias_in_favor_cumsum)
    
    # Step 8: Compare the cumulative bias for +delta and -delta, and choose the one with higher cumulative bias
    final_bias_cumsum = max(bias_in_favor_cumsums, key=lambda x: x[-1])
    
    # Return cumulative bias if `cumulative=True`, otherwise return the final bias value
    if cumulative:
        return final_bias_cumsum
    else:
        return final_bias_cumsum[-1]


#### Example: Hypothesis Bias In Favor

In [79]:
from scipy.stats import uniform

# Parameters
n_sample = 50
hypothesis = 0
delta = 0.5
n_datasets = 1000
n_marginal = 1000
sigma = 1

# Define a uniform prior over the interval [-2, 2]
uniform_prior = uniform(loc=-2, scale=4)  # loc is the start, scale is the width

# Calculate bias in favor using the uniform prior
bias_in_favor_uniform = ss_hypothesis_bias_in_favor_normal_1p(
    n_sample=n_sample,
    hypothesis=hypothesis,
    delta=delta,
    n_datasets=n_datasets,
    n_marginal=n_marginal,
    sigma=sigma,
    prior=uniform_prior,
    cumulative=False
)

print(f"Bias in favor of the hypothesis with uniform prior: {bias_in_favor_uniform}")


Bias in favor of the hypothesis with uniform prior: 0.006


## Estimation

### Estimation Bias Against

In [81]:
import numpy as np
from scipy.stats import norm
from scipy.special import logsumexp

def ss_estimation_bias_against_normal_1p(
    n_sample,
    n_datasets=1000,
    n_marginal=1000,
    sigma=1,
    prior=None,
    cumulative=False
):
    """
    Estimates the bias against a normal parameter estimate using sufficient statistics with an arbitrary prior.

    Parameters:
    ----------
    n_sample : int
        Sample size for each dataset (number of observations).
    n_datasets : int, optional
        Number of datasets (sample means) to simulate (default is 1000).
    n_marginal : int, optional
        Number of samples to estimate the marginal likelihood (default is 1000).
    sigma : float, optional
        Known standard deviation of the data (default is 1).
    prior : scipy.stats distribution, optional
        Prior distribution object (frozen) from scipy.stats. Default is standard normal.
    cumulative : bool, optional
        If True, returns the cumulative bias against (default is False).

    Returns:
    -------
    float or np.ndarray
        Final bias against estimate or cumulative bias against values.
    """
    # Set default prior if none is provided (standard normal)
    if prior is None:
        prior = norm(loc=0, scale=1)

    # Standard error of the mean
    se = sigma / np.sqrt(n_sample)
    
    # Step 1: Sample theta_0 from the prior distribution
    theta_0_samples = prior.rvs(size=n_datasets)
    
    # Step 2: Generate sample means x̄ from p(x̄ | theta_0)
    x_bar = np.random.normal(theta_0_samples, se)
    
    # Step 3: Compute log-likelihoods of x̄ under theta_0
    log_likelihood = norm.logpdf(x_bar, loc=theta_0_samples, scale=se)
    
    # Step 4: Sample theta_j from the prior distribution
    theta_j_samples = prior.rvs(size=n_marginal)
    
    # Step 5: Compute the marginal likelihood p(x̄) via Monte Carlo integration
    # Compute log p(x̄ | theta_j) for each theta_j and x̄
    # Shape of log_likelihoods: (n_datasets, n_marginal)
    log_likelihoods = norm.logpdf(
        x_bar[:, np.newaxis],
        loc=theta_j_samples[np.newaxis, :],
        scale=se
    )
    
    # Compute log marginal likelihood for each x̄ using log-sum-exp trick
    # log p(x̄) = log(1/n_marginal * sum_{j} exp(log p(x̄ | theta_j)))
    log_marginal_likelihood = logsumexp(log_likelihoods, axis=1) - np.log(n_marginal)
    
    # Step 6: Compute the bias against
    bias_against = (log_likelihood - log_marginal_likelihood) <= 0
    
    # Compute cumulative bias against if requested
    bias_against_cumsum = np.cumsum(bias_against) / np.arange(1, n_datasets + 1)
    
    if cumulative:
        return bias_against_cumsum
    else:
        return bias_against_cumsum[-1]


#### Example: Estimation Bias Against

In [175]:
from scipy.stats import uniform

# Parameters
n_sample = 50
n_datasets = 1000
n_marginal = 1000
sigma = 1

# Define a uniform prior over the interval [-2, 2]
uniform_prior = uniform(loc=-2, scale=4)  # loc is the start, scale is the width

# Calculate bias against using the uniform prior
bias_against_uniform = ss_estimation_bias_against_normal_1p(
    n_sample=n_sample,
    n_datasets=n_datasets,
    n_marginal=n_marginal,
    sigma=sigma,
    prior=uniform_prior,
    cumulative=False
)

print(f"Bias against the parameter estimate with uniform prior: {bias_against_uniform}")


Bias against the parameter estimate with uniform prior: 0.006


### Estimation Bias In Favor

In [176]:
import numpy as np
from scipy.stats import norm
from scipy.special import logsumexp

def ss_estimation_bias_in_favor_normal_1p(
    n_sample,      # Sample size of each dataset x
    delta,         # The distance delta
    n_iter=50,     # Number of iterations over theta_0
    sigma=1,       # Known standard deviation of data
    prior=None,    # Prior distribution (frozen scipy.stats object)
    n_marginal=500,  # Number of samples to estimate p(x̄)
    n_datasets=50,    # Number of datasets per theta
    cumulative=False  # Return cumulative bias if True, otherwise return the final bias value
):
    """
    Estimates bias in favor of a normal parameter estimate using sufficient statistics with an arbitrary prior.
    """

    # Set default prior if none is provided (standard normal)
    if prior is None:
        prior = norm(loc=0, scale=1)

    # Standard error of the mean
    se = sigma / np.sqrt(n_sample)
    
    # Pre-sample all theta_0 values from the prior distribution
    theta_0_samples = prior.rvs(size=n_iter)
    
    # Pre-sample theta_j_samples for marginal likelihood estimation
    theta_j_samples = prior.rvs(size=n_marginal)
    
    # Initialize an array to store the maximum biases from each iteration
    max_biases = np.zeros(n_iter)
    
    for i, theta_0 in enumerate(theta_0_samples):
        # Array to store biases for theta = theta_0 + delta and theta_0 - delta
        biases = []
        
        for delta_sign in [+delta, -delta]:
            # Step 1: Set theta = theta_0 + delta or theta_0 - delta
            theta = theta_0 + delta_sign
            
            # Step 2: Generate sample means x̄ from p(x̄ | theta)
            # x_bar shape: (n_datasets,)
            x_bar = np.random.normal(theta, se, size=n_datasets)
            
            # Step 3: Compute log p(x̄ | theta_0) for each dataset
            # log_p_x_bar_given_theta0 shape: (n_datasets,)
            log_p_x_bar_given_theta0 = norm.logpdf(x_bar, loc=theta_0, scale=se)
            
            # Step 4: Estimate log p(x̄) via Monte Carlo integration
            # Compute log p(x̄ | theta_j) for all theta_j and datasets
            # log_p_x_bar_given_theta_j shape: (n_datasets, n_marginal)
            x_bar_expanded = x_bar[:, np.newaxis]  # Shape: (n_datasets, 1)
            theta_j_expanded = theta_j_samples[np.newaxis, :]  # Shape: (1, n_marginal)
            
            log_p_x_bar_given_theta_j = norm.logpdf(x_bar_expanded, loc=theta_j_expanded, scale=se)
            
            # Compute log p(x̄) for each dataset using log-sum-exp trick
            # log_p_x_bar shape: (n_datasets,)
            log_p_x_bar = logsumexp(log_p_x_bar_given_theta_j, axis=1) - np.log(n_marginal)
            
            # Step 5: Compute the log relative belief ratio for each dataset
            log_rb = log_p_x_bar_given_theta0 - log_p_x_bar
            
            # Step 6: Determine bias for each dataset
            biases_datasets = (log_rb > 0).astype(int)
            
            # Step 7: Compute the bias for this theta as the mean over datasets
            bias = biases_datasets.mean()
            
            # Store the bias for this delta sign
            biases.append(bias)
        
        # Step 8: Store the maximum of the two biases
        max_biases[i] = max(biases)
    
    # Step 9: Compute the final bias in favor estimate
    bias_in_favor_estimate = max_biases.mean()
    
    # Return cumulative bias if `cumulative=True`, otherwise return the final bias value
    if cumulative:
        return np.cumsum(max_biases) / np.arange(1, len(max_biases) + 1)
    else:
        return bias_in_favor_estimate


#### Example: Estimation Bias In Favor

In [202]:
from scipy.stats import uniform

# Parameters
n_sample = 50
delta = 0.5
n_iter = 100
sigma = 1

# Define a uniform prior over the interval [-2, 2]
uniform_prior = uniform(loc=-2, scale=4)  # loc is the start, scale is the width

# Calculate bias in favor using the uniform prior
bias_in_favor_uniform = ss_estimation_bias_in_favor_normal_1p(
    n_sample=n_sample,
    delta=delta,
    n_iter=n_iter,
    sigma=sigma,
    prior=uniform_prior,
    n_marginal=500,
    n_datasets=100,
    cumulative=False
)

print(f"Bias in favor of the parameter estimate with uniform prior: {bias_in_favor_uniform}")


Bias in favor of the parameter estimate with uniform prior: 0.18150000000000005
